### IS Proof of concept
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [8]:
from IPython.display import clear_output

In [10]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_nlp_file = root_path+"/comment_cooc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [11]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN"]
col_thread = mydb["NIDA_PPSN_THREAD_V2"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

In [12]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [13]:
def checkThai(txt):
    prop = pythainlp.util.countthai(txt)
    return prop

In [14]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i))
    return returnList

In [15]:
def tagThai(w):
    pd_returnList = pd.DataFrame(pos_tag(w, corpus='pud'))
    returnList = pd_returnList.iloc[:,1]
    return returnList

In [16]:
def soundThai(w):
    returnList = []
    for i in w:
        returnList.append(romanize(i, engine="thai2rom"))
    return returnList

In [17]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

### Clean data & NLP Tokenized

In [16]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [17]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived_date']
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived_date']

In [18]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,1000.0,43.049,53.682105,3.0,11.0,21.0,49.0,351.0


In [19]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

meiji comment: 43049
meiji URLs ก่อนตัด: 1000


In [20]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,517.0,44.756286,57.056773,1.0,10.0,19.0,53.0,328.0


In [21]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

DDCF comment: 23139
DDCF URLs ก่อนตัด: 517


In [22]:
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [23]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 66188
All URLs ก่อนตัด: 1517


In [24]:
missing = df_Corpus.isnull().sum()
print(missing)

Table               0
URLs                0
text             4703
Retrived_date       0
dtype: int64


In [25]:
df_Corpus.tail()

,Table,URLs,text,Retrived_date
66183,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [26]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived_date']

In [27]:
df_Corpus.tail()

,commentId,Table,URLs,text,Retrived_date
66183,66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,66188,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [28]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [29]:
missing = df_Corpus.isnull().sum()
print(missing)

Table            0
URLs             0
text             0
Retrived_date    0
dtype: int64


In [30]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [31]:
df_Corpus.insert(4,'token_text',None)
#df_Corpus.insert(5,'token_sound',None)
#df_Corpus.insert(6,'token_tag',None)
df_Corpus.tail()

,Table,URLs,text,Retrived_date,token_text
commentId,,,,,
66181,DDFC,https://pantip.com/topic/30651064,นมผง คือเอานมดิบดีๆ มาคัดไขมันนมและอื่นๆออกไปใ...,2020-10-27 13:54:33.920930,None
66182,DDFC,https://pantip.com/topic/30651064,นมสด รสชาติมันดีนะ\nเพราะมีปริมาณไขมันสูง ดังน...,2020-10-27 13:54:33.920930,None
66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930,None
66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930,None
66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930,None


เก็บเฉพาะหัวกระทู้

In [32]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','text']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_text',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,text,token_text,mention_product,mention_brand
commentId,,,,,
65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,None,None,None
66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,None,None,None
66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...",None,None,None
66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,None,None,None
66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,None,None,None


## ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [33]:
garbage_char = ['',' ','🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','(',')','ฯ','ๆ','กก','อันนี้','%s','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','://']
garbage_char  = garbage_char + list_punc

In [34]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [35]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','โฟร์โมสต์','โฟร์โมส'
               ,'โชคชัย','แดรี่โฮม','dairy home','อืมม มิลค์','Umm! Milk','เอ็มมิลค์','mmilk','เอ็มมิ้ลค์'
               ,'แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','หนองโพ'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','พาสเจอไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','น้ำตาลมะพร้าว','นมฮอกไกโด'
               ,'เบดไทม์','ฟรีแลคโตส','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต','คาราเมล','ช็อกโกมอลต์','ซื้อไม่ได้','ไม่ได้'
               ,'ทุกคน','ท็อปส์','แมคโคร','โลตัส','บิ๊กซี','bigc','shelf life','ขายไม่ดี','แพคคู่','ค่าจัดส่ง'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
              ]
words = set(thai_words()).union(set(custom_list))
custom_tokenizer = Tokenizer(engine='newmm', custom_dict=words, keep_whitespace=False)

เก็บข้อมูลลง MongoDB

In [37]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "text": df.text[c],
            "token_text":df.token_text[c],
            "mention_product": df.mention_product[c],
            "mention_brand": df.mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [38]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "Table": df.Table[c],
            "URLs": lx,
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c]
        }
        listofdict.append(info)
    return listofdict

In [39]:
def tokenize_custom(x,custom_tokenizer):
    wtkn = custom_tokenizer.word_tokenize(x)
    wtkn = checkStopwords(wtkn)
    wtkn = [x for x in wtkn if x not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [40]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

In [41]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

In [42]:
df_process_comment.head()

,commentId,Table,URLs,text,Retrived_date,token_text
0,1,Meiji,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,2,Meiji,https://pantip.com/topic/39868603,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,3,Meiji,https://pantip.com/topic/39868603,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."
3,4,Meiji,https://pantip.com/topic/39868603,เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, low..."
4,5,Meiji,https://pantip.com/topic/39868603,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่..."


In [43]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

In [44]:
df_process_thread['token_text'] = df_process_thread['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

In [45]:
df_process_thread.tail()

,commentId,URLs,text,token_text,mention_product,mention_brand
1509,65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,"[คุณแม่มือใหม่, ทำตัว, ไม่, ทราบ, ที่จะ, กินนม...",None,None
1510,66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...",None,None
1511,66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...","[เพื่อน, คุ้นเคย, คำถาม, ทุ, กค, น., .., ?"", ห...",None,None
1512,66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,"[แตก, กระทู้, มาจาก, บ้าน, รูป, เฟส, นะจ๊ะ, วั...",None,None
1513,66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,"[อยากรู้, นมกล่อง, ทึ่, ขาย, ห้างสรรพสินค้า, น...",None,None


In [46]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [47]:
del df_textList1,df_textList2,df_thread,df_Corpus,df_process_comment,df_process_thread

Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

In [27]:
def find_brands_product(x,catType):
    ret_wtkn = x
    brands = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','โฟร์โมสต์','โฟร์โมส'
               ,'โชคชัย','แดรี่โฮม','dairy home','อืมม มิลค์','Umm! Milk','เอ็มมิลค์','mmilk','เอ็มมิ้ลค์'
               ,'แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','หนองโพ']
    products = ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','พาสเจอไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','น้ำตาลมะพร้าว'
               ,'เบดไทม์','ฟรีแลคโตส','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต','คาราเมล','ช็อกโกมอลต์']
    for i,each in enumerate(ret_wtkn,0):
        if catType == 'b' and each in brands:
            ret_wtkn[i] = each
        elif catType == 'p' and each in products:
            ret_wtkn[i] = each
        else:
            ret_wtkn[i] = None
    return ret_wtkn

In [48]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [49]:
df_thr_process = df_thr_process[0:5]
df_thr_process

,URLs,_id,mention_brand,mention_product,text,token_text
0,https://pantip.com/topic/39868603,5fb36a76f0f7df4fd532fc98,None,None,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,https://pantip.com/topic/39597709,5fb36a76f0f7df4fd532fc99,None,None,ตามหัวข้อเลยค่ะ อยากรู้ว่านมเมจิที่ต้มก่อนนำไป...,"[หัวข้อ, อยากรู้, นม, เมจิ, ต้ม, ส่วนผสม, เครื..."
2,https://pantip.com/topic/39747201,5fb36a76f0f7df4fd532fc9a,None,None,วันก่อนซื้อ นมเมจิขวด 2 ลิตร มากิน สังเกตฝามัน...,"[วันก่อน, ซื้อ, นม, เมจิ, ขวด, ลิตร, กิน, สังเ..."
3,https://pantip.com/topic/39732615,5fb36a76f0f7df4fd532fc9b,None,None,อยากรู้ว่าMakroมีนมยี่ห้อ meiji(เมจิ) ขายมั้ยครับ,"[อยากรู้, Makro, นม, ยี่ห้อ, meiji, เมจิ, ขาย]"
4,https://pantip.com/topic/39005469,5fb36a76f0f7df4fd532fc9c,None,None,อยากถามว่า\r\n\r\n1.คิดค่าจัดส่งยังไงครับ\r\n2...,"[ถาม, 1., ค่าจัดส่ง, 2., ซื้อ, ได้, ส่วนลด, 3...."


In [50]:
df_thr_process['mention_brand'] = df_thr_process['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_thr_process

,URLs,_id,mention_brand,mention_product,text,token_text
0,https://pantip.com/topic/39868603,5fb36a76f0f7df4fd532fc98,"[None, None, None, None, เมจิ, None, None, Non...",None,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[None, None, None, None, เมจิ, None, None, Non..."
1,https://pantip.com/topic/39597709,5fb36a76f0f7df4fd532fc99,"[None, None, None, เมจิ, None, None, None, Non...",None,ตามหัวข้อเลยค่ะ อยากรู้ว่านมเมจิที่ต้มก่อนนำไป...,"[None, None, None, เมจิ, None, None, None, Non..."
2,https://pantip.com/topic/39747201,5fb36a76f0f7df4fd532fc9a,"[None, None, None, เมจิ, None, None, None, Non...",None,วันก่อนซื้อ นมเมจิขวด 2 ลิตร มากิน สังเกตฝามัน...,"[None, None, None, เมจิ, None, None, None, Non..."
3,https://pantip.com/topic/39732615,5fb36a76f0f7df4fd532fc9b,"[None, None, None, None, None, เมจิ, None]",None,อยากรู้ว่าMakroมีนมยี่ห้อ meiji(เมจิ) ขายมั้ยครับ,"[None, None, None, None, None, เมจิ, None]"
4,https://pantip.com/topic/39005469,5fb36a76f0f7df4fd532fc9c,"[None, None, None, None, None, None, None, Non...",None,อยากถามว่า\r\n\r\n1.คิดค่าจัดส่งยังไงครับ\r\n2...,"[None, None, None, None, None, None, None, Non..."


In [18]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [ ]:
dictionary = gensim.corpora.Dictionary(df_cmt_process['token_text'])
#dictionary.token2id.keys()

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_cmt_process['token_text']]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
num_topics = 10
chunksize = 4000 # size of the doc looked at every pass
passes = 21 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

In [ ]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, lambda_step=0.1)

## สร้าง Bag of word dataframe
ไม่มีทางเลือก Review Lit มาแล้ว ต้องคัดคำระดับ 1000 คำ มาทำ รันไปเลย 4-6 ชั่วโมง รันจบแล้วถึงทำ  Factor Analysis/T-SNE ต่อ<br>
ให้คัดเลือก flavor, brand, market clean ตาม term ให้ครบ นอกนั้นให้ดึงมาทั้งหมดตรงๆ

1. ตัดคำ และ Process คำ รอบที่ 2
2. Clean คำ ชื่อ Flavor และชื่อ Brand
3. นับคำในแต่ละเอกสาร (Comment)
4. รอ รอ รอ

### Create co-occurrence matrix จาก Bag of word

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def cnt_Grpby(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    _PAIB = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return _PAIB.freq.sum()

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_past_nlp_file, index=True)

### Create co-occurrence matrix with Lift normalization

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def a_in_b(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,j].name)[df.iloc[:,i].name].value_counts())
    PAIB.index.names = ['B','A']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    PAIB_ = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return PAIB_.freq.sum()/PAIB.freq.sum()

In [ ]:
for i in range(0,len(item_item_matrix.columns)) :
    for j in range(0,len(item_item_matrix.columns)) :
        A = join_df.iloc[:,i]
        B = join_df.iloc[:,j]
        PA = A[A!=0].count()/A.shape[0]
        PB = B[B!=0].count()/B.shape[0]
        PAB = PB*a_in_b(join_df,i,j)
        item_item_matrix.iloc[i,j] = PAB/(PA*PB)

In [ ]:
item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_past_nlp_file, index=True)